### AI Fairness 360 en Colab

Este notebook brinda un ejemplo básico sobre la herramienta AI Fairness 360 (AIF360) de IBM. Especificamente, se usará una métrica de equidad (mean difference) y un algoritmo de mitigación de sesgo (optimized preprocessing) en el contexto del sesgo de edad. 

El dataset utilizado es [Credit Scoring](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29) (Calificación Crediticia) de German Credit.

* *Para más información sobre este notebook visite este [link](https://nbviewer.org/github/Trusted-AI/AIF360/blob/master/examples/tutorial_credit_scoring.ipynb).*

* *Para más información sobre la herramienta visite este [link](https://aif360.mybluemix.net/)*

#### Librerias

In [ ]:
%pip install -q wget

  Preparing metadata (setup.py) ... done


In [ ]:
# Despues de ejecutar esta celda debes Reiniciar Entorno de Ejecución y nuevamente ejecutar esta celda
%pip install 'aif360[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
import wget
import os
import numpy as np
from IPython.display import Markdown, display
import pandas as pd

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

sys.path.insert(1, "../")
np.random.seed(0)
pd.set_option('display.max_columns', None)

#### AI Fairness 360

En el código base de AIF360, existen tres puntos para detectar y mitigar el sesgo (preprocesamiento - *pre-processing*, procesamiento interno - *in-processing* y procesamiento posterior - *post-processing*).

"Buscaremos sesgos en la creación de un modelo ML para predecir si un solicitante debe recibir crédito en función de varias características de una solicitud de crédito típica. El atributo protegido será Edad (`age`), siendo '1' (mayor o igual a 25 años) y '0' (menor de 25) los valores para los grupos privilegiados y no privilegiados, respectivamente". 

"Verificaremos el sesgo en los datos de entrenamiento inicial, mitigaremos el sesgo y volveremos a verificar".

* *Visite este [link](https://aif360.readthedocs.io/en/stable/) para ver la documentación de AIF360.*

#### Dataset

El conjunto de datos utilizado se puede utilizar para entrenar un modelo el cual predice si un solicitante de prestamo bancario pagará o no.

"*El escenario del préstamo bancario describe un ejemplo intuitivo de sesgo ilegal, ya que el modelo puede basar su decisión en la edad de un solicitante, independientemente de si esta es una buena predicción basada en datos históricos. Sin embargo, no todos los sesgos indeseables en el aprendizaje automático son ilegales; también pueden existir de formas más sutiles. Por ejemplo, una compañía de préstamos puede desear una cartera diversa de clientes en todos los niveles de ingresos y, por lo tanto, considerará indeseable otorgar más préstamos a niveles de ingresos altos que a niveles de ingresos bajos*".

"*Un conjunto de herramientas de detección y/o mitigación de sesgos debe adaptarse al sesgo particular de interés. Más específicamente, necesita conocer el atributo o atributos, denominados atributos protegidos, que son de interés: la raza es un ejemplo de atributo protegido y la edad es un segundo*".

* Visite este [link](https://github.com/jpzambranoleon/german-credit-data-project) para visualizar la base de datos 'procesada'.

In [ ]:
# Descargar y mover archivos necesarios para evitar un error no resuelto en la clase 'GermanDataset'

output_directory = os.path.join("/usr/local/lib/python3.9/dist-packages/aif360/data/raw/german")

german_data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
german_doc_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc"

german_data = wget.download(german_data_url, out=output_directory)
german_doc = wget.download(german_doc_url, out=output_directory)

In [ ]:
dataset = GermanDataset()
dataset, _ = dataset.convert_to_dataframe(de_dummy_code=True, set_category=True)
dataset

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,credit,status,credit_history,purpose,savings,employment,other_debtors,property,installment_plans,housing,skill_level,telephone,foreign_worker
0,6.0,1169.0,4.0,4.0,Old,2.0,1.0,Male,Good Credit,A11,A34,A43,A65,A75,A101,A121,A143,A152,A173,A192,A201
1,48.0,5951.0,2.0,2.0,Young,1.0,1.0,Female,Bad Credit,A12,A32,A43,A61,A73,A101,A121,A143,A152,A173,A191,A201
2,12.0,2096.0,2.0,3.0,Old,1.0,2.0,Male,Good Credit,A14,A34,A46,A61,A74,A101,A121,A143,A152,A172,A191,A201
3,42.0,7882.0,2.0,4.0,Old,1.0,2.0,Male,Good Credit,A11,A32,A42,A61,A74,A103,A122,A143,A153,A173,A191,A201
4,24.0,4870.0,3.0,4.0,Old,2.0,2.0,Male,Bad Credit,A11,A33,A40,A61,A73,A101,A124,A143,A153,A173,A191,A201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12.0,1736.0,3.0,4.0,Old,1.0,1.0,Female,Good Credit,A14,A32,A42,A61,A74,A101,A121,A143,A152,A172,A191,A201
996,30.0,3857.0,4.0,4.0,Old,1.0,1.0,Male,Good Credit,A11,A32,A41,A61,A73,A101,A122,A143,A152,A174,A192,A201
997,12.0,804.0,4.0,4.0,Old,1.0,1.0,Male,Good Credit,A14,A32,A43,A61,A75,A101,A123,A143,A152,A173,A191,A201
998,45.0,1845.0,4.0,4.0,Young,1.0,1.0,Male,Bad Credit,A11,A32,A43,A61,A73,A101,A124,A143,A153,A173,A192,A201


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   month                            1000 non-null   float64 
 1   credit_amount                    1000 non-null   float64 
 2   investment_as_income_percentage  1000 non-null   float64 
 3   residence_since                  1000 non-null   float64 
 4   age                              1000 non-null   object  
 5   number_of_credits                1000 non-null   float64 
 6   people_liable_for                1000 non-null   float64 
 7   sex                              1000 non-null   object  
 8   credit                           1000 non-null   object  
 9   status                           1000 non-null   category
 10  credit_history                   1000 non-null   category
 11  purpose                          1000 non-null   category
 12  savings     

##### **Especificar el atributo protegido**

"El atributo protegido será `age`. Aunque usaremos solo el conjunto de datos de entrenamiento en este tutorial, un flujo de trabajo normal también usaría un conjunto de datos de test para evaluar la eficacia (accuracy, fairness, etc.) durante el desarrollo de un modelo ML. Finalmente, establecemos dos variables (que se usarán más adelante) para los valores privilegiados (1) y no privilegiados (0) para el atributo `age`. Estos son insumos clave para detectar y mitigar el sesgo".

In [ ]:
# Este dataset también contiene atributos protegidos como 'sex' que no consideramos en esta evaluación.
# Se ignoran los atributos/columnas relacionadas con el sexo

dataset_orig = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25],
    features_to_drop=['personal_status', 'sex']
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

# La edad >=25, es decir, con valor 1, se considera privilegiada
privileged_groups = [{'age': 1}]
# La edad <25 (0), es decir, con valor 0 se considera no privilegiada
unprivileged_groups = [{'age': 0}]

#### Calcular la métrica de equidad

"Ahora que identificamos el atributo protegido `age` y definimos valores privilegiados y no privilegiados, podemos usar AIF360 para detectar sesgos en el conjunto de datos. Una prueba sencilla es comparar el porcentaje de resultados favorables para los grupos privilegiados y no privilegiados, restando el primero del segundo". 

"Un valor negativo indica resultados menos favorables para los grupos no privilegiados. Esto se implementa en el método llamado `mean_difference` en la clase `BinaryLabelDatasetMetric`".

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

# Métrica de equidad (mean difference)
metric_f = metric_orig_train.mean_difference()

display(Markdown("#### Dataset de entrenamiento original"))
print("Diferencia en los resultados medios entre grupos privilegiados y no privilegiados = %f" % metric_f)

#### Dataset de entrenamiento original

Diferencia en los resultados medios entre grupos privilegiados y no privilegiados = -0.169905


El método `mean_diference()` hace referencia a `statistical_parity_difference()` y su formula es:

$$ Pr(Y = 1 | D = unprivileged) − Pr(Y = 1 | D = privileged) $$


* Para entender un poco más sobre la elección de métricas y mitigación visite este [link](https://aif360.mybluemix.net/resources#guidance)

#### Mitigar el sesgo

Para mitigar el sesgo se transformará el dataset original. 

"El paso anterior mostró que el grupo privilegiado (personas con 25 años de edad o más) estaba obteniendo un ~17% más de resultados positivos en el conjunto de datos de entrenamiento. Dado que esto no es deseable, vamos a intentar mitigar este sesgo en el conjunto de datos de entrenamiento. Esto se denomina mitigación de preprocesamiento (*pre-processing mitigation*) porque ocurre antes de la creación del modelo".

"AIF360 implementa varios algoritmos de mitigación de preprocesamiento. Elegiremos el algoritmo Reweighing [(Kamiran & Calders, 2012)](https://link.springer.com/article/10.1007/s10115-011-0463-8), que se implementa en la clase `Reweighing` en el paquete `aif360.algorithms.preprocessing`. Este algoritmo transformará el conjunto de datos para que tenga más equidad en los resultados positivos del atributo protegido para los grupos privilegiados y no privilegiados".

"Luego llamamos a los métodos fit y transform (`fit_transform`) para realizar la transformación, produciendo un conjunto de datos de entrenamiento recién transformado (`dataset_transf_train`)".

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)

dataset_transf_train = RW.fit_transform(dataset_orig_train)

#### Calcular la métrica de equidad en el dataset transformado

"Ahora que tenemos un conjunto de datos transformado, podemos verificar qué tan efectivo fue para eliminar el sesgo usando la misma métrica que usamos para el conjunto de datos de entrenamiento original. Una vez más, usamos la función `mean_difference` en la clase `BinaryLabelDatasetMetric`". 

In [ ]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

# Métrica de equidad (mean difference)
metric_f_2 = metric_orig_train.mean_difference()

display(Markdown("#### Dataset de entrenamiento transformado"))
print("Diferencia en los resultados medios entre grupos privilegiados y no privilegiados = %f" % metric_f_2)

#### Dataset de entrenamiento transformado

Diferencia en los resultados medios entre grupos privilegiados y no privilegiados = 0.000000


"Vemos que el paso de mitigación fue muy efectivo, la diferencia en los resultados medios (mean difference) ahora es 0%. Así pasamos de una ventaja de ~17% para el grupo privilegiado a la igualdad en términos de resultados medios".